<a href="https://colab.research.google.com/github/acordobaf/tpp/blob/main/simulacion_tesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

class Chacra:
    def __init__(self, name, gamelas_available, local_available, personal_alquileres, rental_cost, needs):
        self.name = name
        self.gamelas_available = gamelas_available
        self.local_available = local_available
        self.rental_cost = rental_cost
        self.needs = needs
        self.personal_alquileres = personal_alquileres

    def necesidad(self, semana):
        return self.needs[semana - 2]

    def disponibilidad(self):
        return self.gamelas_available + self.local_available

    def balance(self, semana):
        return self.disponibilidad() - self.necesidad(semana)


# Clase Costos
class Costos:
    def __init__(self, sprinter_cost=420, sprinter_capacity=18, bus_cost=1000, bus_capacity=50):
        self.sprinter_cost = sprinter_cost
        self.sprinter_capacity = sprinter_capacity
        self.bus_cost = bus_cost
        self.bus_capacity = bus_capacity

    def calcular_transporte(self, cantidad, work_days=6):
        sprinter_trips = -(-cantidad // self.sprinter_capacity)
        bus_trips = -(-cantidad // self.bus_capacity)
        sprinter_total_cost = sprinter_trips * self.sprinter_cost * work_days
        bus_total_cost = bus_trips * self.bus_cost * work_days
        return min(sprinter_total_cost, bus_total_cost), "Sprinter" if sprinter_total_cost < bus_total_cost else "Bus"

    def calcular_ociosidad(self, superavit, daily_wage=50, work_days=6):
        return superavit * daily_wage * work_days

    def evaluar_opciones(self, chacra_origen, semana, chacras):
        balance_origen = chacra_origen.balance(semana)
        costos = {}
        flujo_personal = []
        opciones = {"Opción 1": None, "Opción 2": None}

        if balance_origen < 0:  # Déficit
            deficit = abs(balance_origen)

            # Calcular capacidad de alquiler y el déficit restante
            capacidad_alquiler = min(deficit, chacra_origen.personal_alquileres)  # Personas que pueden ser cubiertas por alquiler
            costo_alquiler = capacidad_alquiler * chacra_origen.rental_cost / 4 if capacidad_alquiler > 0 else float('inf')

            # Reducir el déficit con el alquiler
            deficit -= capacidad_alquiler
            if capacidad_alquiler > 0:
                opciones["Opción 1"] = "Alquiler"

            # Calcular costo de transporte para el déficit restante
            transporte_costo, transporte_tipo = self.calcular_transporte(deficit) if deficit > 0 else (float('inf'), None)
            costos["Transporte"] = transporte_costo if deficit > 0 else float('inf')
            costos["Alquiler"] = costo_alquiler if capacidad_alquiler > 0 else float('inf')

            # Generar flujos de personal desde chacras con superávit
            if deficit > 0:
                opciones["Opción 2"] = "Transporte"
                for chacra_destino in chacras.values():
                    if chacra_destino == chacra_origen or deficit <= 0:
                        continue
                    balance_destino = chacra_destino.balance(semana)
                    if balance_destino > 0:  # Superávit en chacra destino
                        mover_cantidad = min(balance_destino, deficit)
                        flujo_personal.append({
                            "Semana": semana,
                            "Desde": chacra_destino.name,
                            "Hacia": chacra_origen.name,
                            "Cantidad Movida": mover_cantidad
                        })
                        deficit -= mover_cantidad
                        # Limitar el flujo al superávit disponible
                        chacra_destino.gamelas_available -= mover_cantidad
                        if deficit == 0:
                            break

                # Si aún hay déficit, asumir que el resto proviene de "extra"
                if deficit > 0:
                    flujo_personal.append({
                        "Semana": semana,
                        "Desde": "Extra",
                        "Hacia": chacra_origen.name,
                        "Cantidad Movida": deficit
                    })

        elif balance_origen > 0:  # Superávit
            superavit = balance_origen
            costos["Ociosidad"] = self.calcular_ociosidad(superavit)
            opciones["Opción 1"] = "Ociosidad"

        else:
            return {"Opción": "Ninguna", "Costo": 0, "Detalles": costos, "Flujo": [], **opciones}

        mejor_opcion = min(costos, key=costos.get)
        return {
            "Opción": mejor_opcion,
            "Costo": costos[mejor_opcion],
            "Detalles": costos,
            "Flujo": flujo_personal,
            **opciones
        }


# Crear instancias de las chacras
chacras = {
    'Blanquita': Chacra('Blanquita', 35, 0, 0, 0, [15, 15, 45, 34, 31, 24, 24, 11, 19, 8, 0, 0, 0, 0, 0, 0, 0]),
    'La Irene': Chacra('La Irene', 18, 2, 6, 600, [14, 14, 42, 32, 13, 17, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
    'La Carolina': Chacra('La Carolina', 0, 4, 6, 600, [3, 3, 8, 6, 12, 12, 12, 0, 0, 0, 0, 3, 4, 4, 5, 5, 2]),
    'La Costa': Chacra('La Costa', 0, 5, 12, 2000, [4, 4, 12, 9, 34, 23, 23, 45, 11, 4, 0, 34, 51, 41, 54, 57, 25]),
    'El Mirasol': Chacra('El Mirasol', 0, 4, 0, 0, [8, 8, 24, 19, 22, 30, 28, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]),
    'San Victorio': Chacra('San Victorio', 18, 1, 0, 0, [1, 1, 4, 3, 0, 0, 3, 6, 10, 4, 0, 0, 0, 0, 0, 0, 0]),
    'La Esperanza': Chacra('La Esperanza', 10, 4, 10, 1600, [10, 10, 31, 24, 10, 18, 17, 16, 13, 3, 0, 1, 1, 1, 1, 1, 1])
}

# Crear instancia de la clase Costos
costos_calculador = Costos()

# Almacenar resultados
resultados = []
flujos = []

# Evaluar opciones por semana y chacra
for semana in range(1, 19):
    for chacra in chacras.values():
        resultado_costos = costos_calculador.evaluar_opciones(chacra, semana, chacras)
        resultados.append({
            "Semana": semana,
            "Chacra": chacra.name,
            "Balance": chacra.balance(semana),
            "Opción 1": resultado_costos["Opción 1"],
            "Opción 2": resultado_costos["Opción 2"],
            "Costo Óptimo": resultado_costos["Costo"],
            **resultado_costos["Detalles"]
        })
        flujos.extend(resultado_costos["Flujo"])

# Convertir los resultados a DataFrames
resultados_df = pd.DataFrame(resultados)
flujos_df = pd.DataFrame(flujos)

# Exportar resultados a Excel
with pd.ExcelWriter("Resultados_Logistica_Completa.xlsx") as writer:
    resultados_df.to_excel(writer, sheet_name="Resultados", index=False)
    if not flujos_df.empty:
        flujos_df.to_excel(writer, sheet_name="Flujos de Personal", index=False)

print("Resultados exportados exitosamente: Resultados_Logistica_Completa.xlsx")


Resultados exportados exitosamente: Resultados_Logistica_Completa.xlsx
